In [2]:
import torch
import json
import numpy as np
from pathlib import Path
from datasets import Dataset, Features, Value, ClassLabel
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
import evaluate
from rapidfuzz import fuzz
import json
import unicodedata
import spacy


In [3]:


def remove_similar(text_list, threshold=95):
    """
    Elimina elementos muy similares entre sí.
    threshold: porcentaje de similitud (0-100)
    """
    unique_texts = []
    for text in text_list:
        is_duplicate = False
        for u_text in unique_texts:
            # ratio de similitud de Levenshtein
            if fuzz.ratio(text, u_text) >= threshold:
                is_duplicate = True
                break
        if not is_duplicate:
            unique_texts.append(text)
    return unique_texts

In [4]:

# Cargar spaCy para español e inglés
nlp_es = spacy.load("es_core_news_sm")
nlp_en = spacy.load("en_core_web_sm")


In [5]:

# Ruta del archivo original
FAQ_PATH = "./faqs/faqs.json"
OUTPUT_PATH = "./faqs/faqs_normalized.json"

# Cargar datos
with open(FAQ_PATH, "r", encoding="utf-8") as f:
    faq_data = json.load(f)["faqs"]

def normalize_text(text: str, lang: str = "es") -> str:
    """
    Normaliza el texto:
    - Minusculas
    - Elimina signos de interrogación
    - Lematización
    """
    # Minusculas
    text = text.lower()
    
    # Eliminar signos de interrogación
    text = text.replace("¿", "").replace("?", "")
    
    # Minúsculas y eliminar tildes
    text = text.lower()
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )

    # Lematización
    nlp = nlp_es if lang == "es" else nlp_en
    doc = nlp(text)
    lemmatized = " ".join([token.lemma_ for token in doc])
    
    # Eliminar espacios dobles
    lemmatized = " ".join(lemmatized.split())
    
    return lemmatized

# Procesar todas las preguntas
for faq in faq_data:
    for lang in ["es", "en"]:
        if lang in faq["questions"]:
            # Normalización previa (minúsculas, lemmatización, quitar signos)
            normalized = [normalize_text(q, lang) for q in faq["questions"][lang]]
            
            # Eliminar duplicados exactos
            normalized = list(dict.fromkeys(normalized))
            
            # Eliminar duplicados similares >95%
            normalized = remove_similar(normalized, threshold=95)
            
            faq["questions"][lang] = normalized

# Guardar JSON normalizado
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump({"faqs": faq_data}, f, ensure_ascii=False, indent=4)

print(f"Archivo normalizado guardado en {OUTPUT_PATH}")


Archivo normalizado guardado en ./faqs/faqs_normalized.json


In [6]:

# =========================================================
# 1. Cargar FAQs
# =========================================================
FAQ_PATH =  "./faqs/faqs_normalized.json"

with open(FAQ_PATH, "r", encoding="utf-8") as f:
    faq_data = json.load(f)["faqs"]

# Preparamos preguntas y etiquetas
questions = []
labels = []
label2id = {}
id2label = {}

# Crear mappings y expandir todas las preguntas en español e inglés
for idx, faq in enumerate(faq_data):
    label2id[str(idx)] = faq["id"]
    id2label[str(faq["id"])] = idx

    # Detectar formato: viejo ("question") o nuevo ("questions")
    q_data = faq.get("questions") or faq.get("question")

    for lang in ["es", "en"]:
        if lang in q_data:
            # Si es string (viejo formato), convertir a lista
            q_list = q_data[lang] if isinstance(q_data[lang], list) else [q_data[lang]]

            for q in q_list:
                questions.append(q)
                labels.append(idx)

In [ ]:


# =========================================================
# 2. Tokenización
# =========================================================
MODEL_NAME = "bert-base-multilingual-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=64
    )

num_classes = len(set(labels))
features = Features({
    "text": Value("string"),
    "label": ClassLabel(num_classes=num_classes)
})

# =========================================================
# 3. Crear dataset con estratificación
# =========================================================
X_train, X_test, y_train, y_test = train_test_split(
    questions,
    labels,
    test_size=0.2,
    stratify=labels,      # <- estratificación por clase
    random_state=42
)

train_data = Dataset.from_dict({"text": X_train, "label": y_train}, features=features)
test_data  = Dataset.from_dict({"text": X_test, "label": y_test}, features=features)

# Tokenizar datasets
train_dataset = train_data.map(tokenize, batched=True)
test_dataset  = test_data.map(tokenize, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# =========================================================
# 4. Definir modelo
# =========================================================
# -------- 4. Modelo --------
num_labels = num_classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # <-- detectar GPU

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels
)

# -------- 5. Configuración de entrenamiento --------
training_args = TrainingArguments(
    output_dir="./faq_model_2",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs_2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,   # <-- aquí aumenté de 3 a 8 épocas
    weight_decay=0.01,
    load_best_model_at_end=True
)


# Métricas
accuracy = evaluate.load("accuracy")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return accuracy.compute(predictions=preds, references=p.label_ids)

# =========================================================
# 6. Entrenador
# =========================================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)




c:\Users\ASUS\anaconda3\envs\torch_env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1061 [00:00<?, ? examples/s]

Map:   0%|          | 0/266 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [62]:
# =========================================================
# 7. Entrenar modelo
# =========================================================
trainer.train()

# =========================================================
# 8. Guardar modelo entrenado
# =========================================================
model.save_pretrained("./faq_model_2")
tokenizer.save_pretrained("./faq_model_2")

  0%|          | 0/536 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

{'eval_loss': 3.291703939437866, 'eval_accuracy': 0.22180451127819548, 'eval_runtime': 11.5192, 'eval_samples_per_second': 23.092, 'eval_steps_per_second': 1.476, 'epoch': 1.0}


  0%|          | 0/17 [00:00<?, ?it/s]

{'eval_loss': 2.5876176357269287, 'eval_accuracy': 0.5977443609022557, 'eval_runtime': 10.635, 'eval_samples_per_second': 25.012, 'eval_steps_per_second': 1.598, 'epoch': 2.0}


...

  0%|          | 0/17 [00:00<?, ?it/s]

{'eval_loss': 1.0533047914505005, 'eval_accuracy': 0.8834586466165414, 'eval_runtime': 10.5794, 'eval_samples_per_second': 25.143, 'eval_steps_per_second': 1.607, 'epoch': 7.0}
{'loss': 2.0333, 'grad_norm': 7.578559875488281, 'learning_rate': 1.3432835820895524e-06, 'epoch': 7.46}


  0%|          | 0/17 [00:00<?, ?it/s]

{'eval_loss': 1.0237741470336914, 'eval_accuracy': 0.8796992481203008, 'eval_runtime': 8.8997, 'eval_samples_per_second': 29.889, 'eval_steps_per_second': 1.91, 'epoch': 8.0}
{'train_runtime': 2878.1513, 'train_samples_per_second': 2.949, 'train_steps_per_second': 0.186, 'train_loss': 1.9653314405412816, 'epoch': 8.0}


('./faq_model_2\\tokenizer_config.json',
 './faq_model_2\\special_tokens_map.json',
 './faq_model_2\\vocab.txt',
 './faq_model_2\\added_tokens.json',
 './faq_model_2\\tokenizer.json')

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# --- Cargar spacy para lematización ---
nlp_es = spacy.load("es_core_news_sm")  # español
nlp_en = spacy.load("en_core_web_sm")   # inglés

# --- Función de normalización ---
def normalize_text(text: str, lang: str = "es") -> str:
    """
    Normaliza el texto:
    - Pasa a minúsculas
    - Elimina signos de interrogación
    - Elimina tildes
    - Lematiza
    """
    # Minusculas
    text = text.lower()
    
    # Eliminar signos de interrogación
    text = text.replace("¿", "").replace("?", "")
    
    # Eliminar tildes
    text = ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )

    # Lematización
    nlp = nlp_es if lang == "es" else nlp_en
    doc = nlp(text)
    lemmatized = " ".join([token.lemma_ for token in doc])
    
    # Quitar espacios dobles
    lemmatized = " ".join(lemmatized.split())
    
    return lemmatized


# --- Cargar modelo y tokenizer entrenados ---
model = AutoModelForSequenceClassification.from_pretrained("./faq_model_2")
tokenizer = AutoTokenizer.from_pretrained("./faq_model_2")

# --- Crear pipeline ---
clf = pipeline(
    "text-classification", 
    model=model, 
    tokenizer=tokenizer, 
    device=0 if torch.cuda.is_available() else -1
)

# --- Función de respuesta con normalización ---
def get_answer(user_question: str, lang: str = "es"):
    # Normalizar antes de pasar al modelo
    clean_q = normalize_text(user_question, lang=lang)
    
    # Obtener predicción
    pred = clf(clean_q, truncation=True, max_length=64)[0]
    
    # Extraer etiqueta y confianza
    label_idx = int(pred["label"].replace("LABEL_", ""))
    confidence = pred["score"]
    
    # Recuperar el ID original de FAQ
    faq_id = label2id[str(label_idx)]
    print(label2id)
    print(id2label)
    # Buscar respuesta en JSON
    for faq in faq_data:
        if faq["id"] == faq_id:
            return faq.get("answer", "No tengo respuesta registrada."), confidence
    
    return "No encontré respuesta.", confidence


# =========================================================
# Ejemplo de uso
# =========================================================
# 15%
pregunta = "que es el SP 500"
respuesta, confianza = get_answer(pregunta, lang="es")

print(f"Pregunta original: {pregunta}")
print(f"Respuesta: {respuesta}")
print(f"Confianza: {confianza:.2%}")

DEBUG Prediction: [{'label': 'LABEL_23', 'score': 0.4423360526561737}]
{'0': 1, '1': 2, '2': 4, '3': 6, '4': 9, '5': 10, '6': 11, '7': 12, '8': 15, '9': 17, '10': 26, '11': 28, '12': 30, '13': 33, '14': 36, '15': 37, '16': 41, '17': 53, '18': 54, '19': 55, '20': 62, '21': 67, '22': 69, '23': 72, '24': 100, '25': 101, '26': 102, '27': 605, '28': 200, '29': 201, '30': 202, '31': 203, '32': 204, '33': 300, '34': 301, '35': 400, '36': 401, '37': 402, '38': 403, '39': 500, '40': 503, '41': 504, '42': 600}
{'1': 0, '2': 1, '4': 2, '6': 3, '9': 4, '10': 5, '11': 6, '12': 7, '15': 8, '17': 9, '26': 10, '28': 11, '30': 12, '33': 13, '36': 14, '37': 15, '41': 16, '53': 17, '54': 18, '55': 19, '62': 20, '67': 21, '69': 22, '72': 23, '100': 24, '101': 25, '102': 26, '605': 27, '200': 28, '201': 29, '202': 30, '203': 31, '204': 32, '300': 33, '301': 34, '400': 35, '401': 36, '402': 37, '403': 38, '500': 39, '503': 40, '504': 41, '600': 42}
Pregunta original: que es el SP 500
Respuesta: {'es': 'El S